### Initial package imports

In [ ]:
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import Dict, List
import random
from matplotlib.gridspec import GridSpec

# Validation files

In [ ]:
validation_set_closing_prices = pd.read_csv('Data/a5/data used/valid data/stocks.csv', sep=',')
display(validation_set_closing_prices.tail())
currency_validation_set = pd.read_csv('Data/a5/data used/origin data/closing_prices_native_currency.csv', sep=';')
currency_validation_df = currency_validation_set[['Date', 'USD/EUR', 'JPY/EUR']]
display(currency_validation_df.tail())
validation_set_closing_prices['Date'] = pd.to_datetime(validation_set_closing_prices['Date'], format='%Y/%m/%d')
# validation_set_closing_prices.set_index('Date', inplace=True)

In [ ]:
# Create log returns for validation set
# For AAPL and MSFT (USD)
validation_set_closing_prices['AAPL_Returns'] = np.log(validation_set_closing_prices['AAPL(USD)']) - np.log(validation_set_closing_prices['AAPL(USD)'].shift(1))
validation_set_closing_prices['MSFT_Returns'] = np.log(validation_set_closing_prices['MSFT(USD)']) - np.log(validation_set_closing_prices['MSFT(USD)'].shift(1))

# For ASML.AS and VOW3.DE (EUR)
validation_set_closing_prices['ASML_Returns'] = np.log(validation_set_closing_prices['ASML.AS(EUR)']) - np.log(validation_set_closing_prices['ASML.AS(EUR)'].shift(1))
validation_set_closing_prices['VOW3_Returns'] = np.log(validation_set_closing_prices['VOW3.DE(EUR)']) - np.log(validation_set_closing_prices['VOW3.DE(EUR)'].shift(1))

# For 6758.T (JPY)
validation_set_closing_prices['Sony_Returns'] = np.log(validation_set_closing_prices['6758.T(JPY)']) - np.log(validation_set_closing_prices['6758.T(JPY)'].shift(1))

validation_set_closing_prices['Return_USD/EUR'] = np.log(currency_validation_df['USD/EUR']) - np.log(currency_validation_df['USD/EUR'].shift(1))
validation_set_closing_prices['Return_JPY/EUR'] = np.log(currency_validation_df['JPY/EUR']) - np.log(currency_validation_df['JPY/EUR'].shift(1))

# Corrected returns
validation_set_closing_prices['C_AAPL_Returns'] = validation_set_closing_prices['AAPL_Returns'] - validation_set_closing_prices['Return_USD/EUR']
validation_set_closing_prices['C_MSFT_Returns'] = validation_set_closing_prices['MSFT_Returns'] - validation_set_closing_prices['Return_USD/EUR']
validation_set_closing_prices['C_ASML_Returns'] = validation_set_closing_prices['ASML_Returns']
validation_set_closing_prices['C_VOW3_Returns'] = validation_set_closing_prices['VOW3_Returns'] 
validation_set_closing_prices['C_Sony_Returns'] = validation_set_closing_prices['Sony_Returns'] - validation_set_closing_prices['Return_JPY/EUR']

display(validation_set_closing_prices.tail())


In [ ]:
weights_validation = {
    'AAPL': 0.2,
    'MSFT': 0.2,
    'ASML.AS': 0.2,
    'VOW3.DE': 0.2,
    '6758.T': 0.2
}

initial_validation_investment = 1000000  # 1 million euros
equity_validation_investment = 600000
loan_validation_investment = 400000

starting_date = '06/04/2010'

starting_row_val = currency_validation_df[currency_validation_df['Date'] == starting_date]

validation_USD_TO_EUR = float(starting_row_val['USD/EUR'].iloc[0])
validation_JPY_TO_EUR = float(starting_row_val['JPY/EUR'].iloc[0])


loan_validation_set = pd.read_csv('Data/a5/data used/valid data/euribor_daily_rates.csv', sep=';')

#replace all the commas with dots
loan_validation_set = loan_validation_set.replace(',', '.', regex=True)
loan_validation_set['3m'] = pd.to_numeric(loan_validation_set['3m'])

year = 360
credit_spread = 0.015

# Calculate daily rate for each row
loan_validation_set['Daily_rate'] = ((loan_validation_set['3m'] / 100) + credit_spread) / year

validation_set_closing_prices['Loan_expenses'] = loan_validation_set['Daily_rate'].iloc[0] * loan_validation_investment

# Set initial investments for first row (index 0)

validation_set_closing_prices = (
    validation_set_closing_prices
    .reset_index(drop=True)    # make the index 0,1,2,...
)


validation_set_closing_prices.loc[0, 'AAPL_Investment'] = initial_validation_investment * weights_validation['AAPL']
validation_set_closing_prices.loc[0, 'MSFT_Investment'] = initial_validation_investment * weights_validation['MSFT']
validation_set_closing_prices.loc[0, 'ASML_Investment'] = initial_validation_investment * weights_validation['ASML.AS']
validation_set_closing_prices.loc[0, 'VOW3_Investment'] = initial_validation_investment * weights_validation['VOW3.DE']
validation_set_closing_prices.loc[0, 'Sony_Investment'] = initial_validation_investment * weights_validation['6758.T']


for i in range(1, len(validation_set_closing_prices)):
    validation_set_closing_prices.loc[i, 'AAPL_Investment'] = \
    validation_set_closing_prices.loc[i-1, 'AAPL_Investment'] \
    * np.exp(validation_set_closing_prices.loc[i, 'C_AAPL_Returns'])
    validation_set_closing_prices.loc[i, 'MSFT_Investment'] = \
    validation_set_closing_prices.loc[i-1, 'MSFT_Investment'] \
    * np.exp(validation_set_closing_prices.loc[i, 'C_MSFT_Returns'])
    validation_set_closing_prices.loc[i, 'ASML_Investment'] = \
    validation_set_closing_prices.loc[i-1, 'ASML_Investment'] \
    * np.exp(validation_set_closing_prices.loc[i, 'C_ASML_Returns'])
    validation_set_closing_prices.loc[i, 'VOW3_Investment'] = \
    validation_set_closing_prices.loc[i-1, 'VOW3_Investment'] \
    * np.exp(validation_set_closing_prices.loc[i, 'C_VOW3_Returns'])
    validation_set_closing_prices.loc[i, 'Sony_Investment'] = \
    validation_set_closing_prices.loc[i-1, 'Sony_Investment'] \
    * np.exp(validation_set_closing_prices.loc[i, 'C_Sony_Returns'])

 
# build portfolio daily log‐return
validation_set_closing_prices['r_portfolio'] = (
      weights_validation['AAPL']    * validation_set_closing_prices['C_AAPL_Returns']
    + weights_validation['MSFT']    * validation_set_closing_prices['C_MSFT_Returns']
    + weights_validation['ASML.AS'] * validation_set_closing_prices['C_ASML_Returns']
    + weights_validation['VOW3.DE'] * validation_set_closing_prices['C_VOW3_Returns']
    + weights_validation['6758.T']  * validation_set_closing_prices['C_Sony_Returns']
)

# 1) make sure the columns exist (as floats) before your loop
validation_set_closing_prices['Total_investment'] = np.nan
validation_set_closing_prices['Total_investment_loan_correction'] = np.nan

# 2) set your initial values at row 0
validation_set_closing_prices.loc[0, 'Total_investment'] = initial_validation_investment
validation_set_closing_prices.loc[0, 'Total_investment_loan_correction'] = (
    initial_validation_investment - validation_set_closing_prices.loc[0, 'Loan_expenses']
)

# 3) now your loop will work without KeyErrors
for i in range(1, len(validation_set_closing_prices)):
    prev_val = validation_set_closing_prices.loc[i-1, 'Total_investment']
    r       = validation_set_closing_prices.loc[i,   'r_portfolio']
    new_val  = prev_val * np.exp(r)
    validation_set_closing_prices.loc[i, 'Total_investment'] = new_val

for i in range(1, len(validation_set_closing_prices)):
    # subtract that day’s loan expense
    prev_val = validation_set_closing_prices.loc[i-1, 'Total_investment_loan_correction']
    r = validation_set_closing_prices.loc[i, 'r_portfolio']
    exp = validation_set_closing_prices.loc[i, 'Loan_expenses']
    corrected_val = prev_val * np.exp(r) - exp
    validation_set_closing_prices.loc[i, 'Total_investment_loan_correction'] = corrected_val

# cols=['Total_investment', 'Total_investment_loan_correction']
# validation_set_closing_prices[cols].tail()



In [ ]:
#plot the total portfolio returns over time
plt.figure(figsize=(15, 7))
plt.plot(validation_set_closing_prices['Date'], validation_set_closing_prices['Total_investment'], label='Total Investment', color='blue')
plt.plot(validation_set_closing_prices['Date'], validation_set_closing_prices['Total_investment_loan_correction'], label='Total Investment with Loan Correction', color='red')
plt.title('Total Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (EUR)')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:

#Calculate the loss per day per asset which is the difference between the investment of the previous day and the current day
validation_set_closing_prices['AAPL_Loss'] = validation_set_closing_prices['Total_investment'] * weights_validation['AAPL'] * validation_set_closing_prices['C_AAPL_Returns'] * -1
validation_set_closing_prices['MSFT_Loss'] = validation_set_closing_prices['Total_investment'] * weights_validation['MSFT'] * validation_set_closing_prices['C_MSFT_Returns'] * -1
validation_set_closing_prices['ASML_Loss'] = validation_set_closing_prices['Total_investment'] * weights_validation['ASML.AS'] * validation_set_closing_prices['C_ASML_Returns'] * -1
validation_set_closing_prices['VOW3_Loss'] = validation_set_closing_prices['Total_investment'] * weights_validation['VOW3.DE'] * validation_set_closing_prices['C_VOW3_Returns'] *-1
validation_set_closing_prices['Sony_Loss'] = validation_set_closing_prices['Total_investment'] * weights_validation['6758.T'] * validation_set_closing_prices['C_Sony_Returns'] *-1

validation_set_closing_prices['MSFT_Loss_loan_correction'] = validation_set_closing_prices['Total_investment_loan_correction'] * weights_validation['MSFT'] * validation_set_closing_prices['C_MSFT_Returns'] *-1
validation_set_closing_prices['AAPL_Loss_loan_correction'] = validation_set_closing_prices['Total_investment_loan_correction'] * weights_validation['AAPL'] * validation_set_closing_prices['C_AAPL_Returns'] *-1
validation_set_closing_prices['ASML_Loss_loan_correction'] = validation_set_closing_prices['Total_investment_loan_correction'] * weights_validation['ASML.AS'] * validation_set_closing_prices['C_ASML_Returns'] *-1
validation_set_closing_prices['VOW3_Loss_loan_correction'] = validation_set_closing_prices['Total_investment_loan_correction'] * weights_validation['VOW3.DE'] * validation_set_closing_prices['C_VOW3_Returns'] *-1
validation_set_closing_prices['Sony_Loss_loan_correction'] = validation_set_closing_prices['Total_investment_loan_correction'] * weights_validation['6758.T'] * validation_set_closing_prices['C_Sony_Returns'] *-1

#calculate the total loss per day
validation_set_closing_prices['Portfolio_Loss'] = (validation_set_closing_prices['AAPL_Loss'] + validation_set_closing_prices['MSFT_Loss'] + validation_set_closing_prices['ASML_Loss'] + validation_set_closing_prices['VOW3_Loss'] + validation_set_closing_prices['Sony_Loss']+ validation_set_closing_prices['Loan_expenses'])
validation_set_closing_prices['Portfolio_Loss_loan_correction'] = (validation_set_closing_prices['AAPL_Loss_loan_correction'] + validation_set_closing_prices['MSFT_Loss_loan_correction'] + validation_set_closing_prices['ASML_Loss_loan_correction'] + validation_set_closing_prices['VOW3_Loss_loan_correction'] + validation_set_closing_prices['Sony_Loss_loan_correction']+ validation_set_closing_prices['Loan_expenses'])

# Calculate the total portfolio value and return
validation_set_closing_prices['Total_Portfolio_Return'] = (validation_set_closing_prices['Total_investment'].pct_change(fill_method=None))
validation_set_closing_prices['Total_Portfolio_Return'] = validation_set_closing_prices['Total_Portfolio_Return'].fillna(0)

validation_set_closing_prices['Total_Portfolio_Return_loan_correction'] = (validation_set_closing_prices['Total_investment_loan_correction'].pct_change(fill_method=None))
validation_set_closing_prices['Total_Portfolio_Return_loan_correction'] = validation_set_closing_prices['Total_Portfolio_Return_loan_correction'].fillna(0)

display(validation_set_closing_prices.tail(10))

In [ ]:
#make a plot with the portfolio loss and portfolio loss loan correction
plt.figure(figsize=(15, 7))
plt.plot(validation_set_closing_prices['Date'], validation_set_closing_prices['Portfolio_Loss'], label='Portfolio Loss', color='blue')
plt.plot(validation_set_closing_prices['Date'], validation_set_closing_prices['Portfolio_Loss_loan_correction'], label='Portfolio Loss with Loan Correction', color='red')
plt.title('Portfolio Loss Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Loss (EUR)')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

# Methods input values 

### Portfolio change Daily

In [ ]:
# Replace instances of main_df with validation_set_closing_prices
if 'Portfolio_Daily_Returns' not in validation_set_closing_prices.columns:
    # Calculate the daily returns from existing return columns
    validation_set_closing_prices['Portfolio_Daily_Returns'] = (
        0.2 * validation_set_closing_prices['C_AAPL_Returns'] +
        0.2 * validation_set_closing_prices['C_MSFT_Returns'] +
        0.2 * validation_set_closing_prices['C_ASML_Returns'] +
        0.2 * validation_set_closing_prices['C_VOW3_Returns'] +
        0.2 * validation_set_closing_prices['C_Sony_Returns']
    )

In [ ]:
def calculate_daily_loss_variables(time_window, current_date):
    # Calculate the mean and standard deviation of portfolio loss from the time windows
    loss_dict = {
        "Date": current_date,
        "Portfolio_mean_loss": np.nanmean(time_window['Portfolio_Loss_loan_correction']),
        "Portfolio_std_loss": np.nanstd(time_window['Portfolio_Loss_loan_correction'])
    }
    return loss_dict

In [ ]:
returns_df = validation_set_closing_prices[['Date', 'C_AAPL_Returns', 'C_MSFT_Returns', 
                                            'C_ASML_Returns', 'C_VOW3_Returns', 'C_Sony_Returns']].dropna()
# set Date as index
returns_df.set_index('Date', inplace=True)

# var_df, es_df = rolling_fhs_multiday_var_es(
#     returns_df,
#     weights_validation,
#     window_size=250,
#     horizons=[1],
#     confidence_levels=[0.95, 0.99],
#     n_simulations=10000,
#     lambdas=[0.94, 0.97],
#     random_seed=42
# )

# # Calculate monetary values for VaR and ES
# monetary_var_df, monetary_es_df = calculate_monetary_values(var_df, es_df, validation_set_closing_prices)
# # Display the monetary VaR and ES DataFrames
# display(monetary_var_df.tail())
# display(monetary_es_df.tail())